In [1]:
import numpy as np
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd /content/gdrive/MyDrive/SLU Semesters/SLU 3rd Semester/NLP/Fourth Competition
%ls 

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/MyDrive/SLU Semesters/SLU 3rd Semester/NLP/Fourth Competition
20epoch_50batch/     cwe-test.txt   sw-test.txt
Anything_goes.ipynb  cwe-train.txt  sw-train.txt


In [3]:
GRAMS_NUM = 6
TRAIN_PATH = "cwe-train.txt"
TEST_PATH = "cwe-test.txt"

In [4]:
training_file = open(TRAIN_PATH, "r")
all_words = training_file.read().split(" ")
splitted = []

for line in all_words:
  splitted.append(line.lower().strip())

train = ' '.join(splitted[:int(len(splitted) * 0.8)])
test = ' '.join(splitted[int(len(splitted) * 0.8):])

In [5]:
len(train)

482816

In [6]:
test_file = open(TEST_PATH, "r")

all_test_words = test_file.read().split(" ")
splitted_test = []

for line in all_test_words:
  splitted_test.append(line.lower().strip())

final_test = ' '.join(splitted_test)

In [7]:
# Source: https://www.analyticsvidhya.com/blog/2019/08/comprehensive-guide-language-model-nlp-python-code/#h2_7
# create a character mapping index
def encode_seq(seq, data_new):
    chars = sorted(list(set(data_new)))
    mapping = dict((c, i) for i, c in enumerate(chars))
    sequences = list()
    for line in seq:
        # integer encode line
        encoded_seq = [mapping[char] for char in line]
        # store
        sequences.append(encoded_seq)
    return sequences, mapping

In [8]:
# Source: https://www.analyticsvidhya.com/blog/2019/08/comprehensive-guide-language-model-nlp-python-code/#h2_7
# define model
def build_model(vocab):
    model = Sequential()
    model.add(Embedding(vocab, 20, input_length= GRAMS_NUM -1, trainable=True))
    model.add(GRU(25, recurrent_dropout=0.1, dropout=0.1))
    model.add(Dense(vocab, activation='softmax'))

    model.compile(loss='categorical_crossentropy', metrics=['acc'], optimizer='adam')

    return model

In [9]:
# Source: https://www.analyticsvidhya.com/blog/2019/08/comprehensive-guide-language-model-nlp-python-code/#h2_7
data = train

n_grams = [''.join(data[i:i + GRAMS_NUM]) for i in (range(len(data) - GRAMS_NUM + 1))]
n_grams, mapping = encode_seq(n_grams, data)

vocab = len(mapping)
sequences = np.array(n_grams)

In [10]:
# Source: https://www.analyticsvidhya.com/blog/2019/08/comprehensive-guide-language-model-nlp-python-code/#h2_7
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.models import Sequential
import keras
from keras.layers import LSTM, Dense, GRU, Embedding
from keras.callbacks import EarlyStopping, ModelCheckpoint
import tensorflow as tf

# create X and y
x, y = sequences[:, :-1], sequences[:, -1]
# one hot encode y
y = to_categorical(y, num_classes=vocab)
# create train and validation sets
x_tr, x_val, y_tr, y_val = train_test_split(x, y, test_size=0.1, random_state=42)

print('Train shape:', x_tr.shape, 'Val shape:', x_val.shape)

model = build_model(vocab)
# https://keras.io/api/callbacks/early_stopping/
callback = tf.keras.callbacks.EarlyStopping(monitor='val_acc', patience=5)

model.fit(x_tr, y_tr, epochs= 20, verbose=1, validation_data=(x_val, y_val), callbacks= callback, batch_size= 50)
model.save('./20epoch_50batch')

Train shape: (434529, 5) Val shape: (48282, 5)


In [11]:
# Source: https://www.analyticsvidhya.com/blog/2019/08/comprehensive-guide-language-model-nlp-python-code/#h2_7
GRAMS_NUM = 6
n_grams = [''.join(final_test[i:i+ GRAMS_NUM]) for i in (range(len(final_test) - GRAMS_NUM + 1))]
n_grams, mapping = encode_seq(n_grams, final_test)

vocab = len(mapping)
sequences = np.array(n_grams)

x_test, y_test = sequences[:, :-1], sequences[:, -1]
# one hot encode y
y_test = to_categorical(y_test, num_classes=vocab)

In [12]:
from math import log2

model = keras.models.load_model('./20epoch_50batch')
predict = model.predict(x_test)

entropy = 0

for pr in predict:
  entropy = entropy -  (1/(len(predict))) * log2(max(pr))

1929/1929 [==============================] - 5s 2ms/step


In [13]:
entropy

1.1953847665600932